In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly as plt
import plotly.graph_objects as go
from sklearn.neighbors import NearestNeighbors
from sklearn import preprocessing
from scipy import spatial 
import warnings
warnings.filterwarnings('ignore')
from plotly.offline import init_notebook_mode, iplot, plot
from plotly.subplots import make_subplots
import plotly.express as px
import plotly as py
init_notebook_mode(connected=True)
import plotly.graph_objs as go

In [2]:
fifa_22 = pd.read_csv(r"D:/Projects/FIFA Recommendation System/archive/players_22.csv")

In [3]:
fifa_22.head()

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,...,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,78000000.0,320000.0,34,...,50+3,50+3,50+3,61+3,19+3,https://cdn.sofifa.net/players/158/023/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png
1,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,92,92,119500000.0,270000.0,32,...,60+3,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/22_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1353/60.png,https://cdn.sofifa.net/flags/pl.png
2,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",91,91,45000000.0,270000.0,36,...,53+3,53+3,53+3,60+3,20+3,https://cdn.sofifa.net/players/020/801/22_120.png,https://cdn.sofifa.net/teams/11/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png
3,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",91,91,129000000.0,270000.0,29,...,50+3,50+3,50+3,62+3,20+3,https://cdn.sofifa.net/players/190/871/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,NaN,https://cdn.sofifa.net/flags/br.png
4,192985,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,91,125500000.0,350000.0,30,...,69+3,69+3,69+3,75+3,21+3,https://cdn.sofifa.net/players/192/985/22_120.png,https://cdn.sofifa.net/teams/10/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png


In [4]:
fifa_22 = fifa_22.iloc[:,0:78]

In [5]:
fifa_22.columns

Index(['sofifa_id', 'player_url', 'short_name', 'long_name',
       'player_positions', 'overall', 'potential', 'value_eur', 'wage_eur',
       'age', 'dob', 'height_cm', 'weight_kg', 'club_team_id', 'club_name',
       'league_name', 'league_level', 'club_position', 'club_jersey_number',
       'club_loaned_from', 'club_joined', 'club_contract_valid_until',
       'nationality_id', 'nationality_name', 'nation_team_id',
       'nation_position', 'nation_jersey_number', 'preferred_foot',
       'weak_foot', 'skill_moves', 'international_reputation', 'work_rate',
       'body_type', 'real_face', 'release_clause_eur', 'player_tags',
       'player_traits', 'pace', 'shooting', 'passing', 'dribbling',
       'defending', 'physic', 'attacking_crossing', 'attacking_finishing',
       'attacking_heading_accuracy', 'attacking_short_passing',
       'attacking_volleys', 'skill_dribbling', 'skill_curve',
       'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control',
       'movement_acce

In [6]:
columns = ['club_team_id','dob','player_positions','player_url','long_name','club_jersey_number','club_loaned_from', 'club_joined', 'club_contract_valid_until',
       'nationality_id','nation_team_id','real_face','player_tags','player_traits']

In [7]:
fifa_22 = fifa_22.drop(columns = columns)

In [8]:
fifa_22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Data columns (total 64 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   sofifa_id                    19239 non-null  int64  
 1   short_name                   19239 non-null  object 
 2   overall                      19239 non-null  int64  
 3   potential                    19239 non-null  int64  
 4   value_eur                    19165 non-null  float64
 5   wage_eur                     19178 non-null  float64
 6   age                          19239 non-null  int64  
 7   height_cm                    19239 non-null  int64  
 8   weight_kg                    19239 non-null  int64  
 9   club_name                    19178 non-null  object 
 10  league_name                  19178 non-null  object 
 11  league_level                 19178 non-null  float64
 12  club_position                19178 non-null  object 
 13  nationality_name

In [9]:
fifa_22.describe()

,sofifa_id,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,league_level,nation_jersey_number,...,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed
count,19239.000000,19239.000000,19239.000000,1.916500e+04,19178.000000,19239.000000,19239.000000,19239.000000,19178.000000,759.000000,...,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,2132.000000
mean,231468.086959,65.772182,71.079370,2.850452e+06,9017.989363,25.210822,181.299704,74.943032,1.354364,12.567852,...,57.929830,46.601746,48.045584,45.906700,16.406102,16.192474,16.055356,16.229274,16.491814,36.439962
std,27039.717497,6.880232,6.086213,7.613700e+06,19470.176724,4.748235,6.863179,7.069434,0.747865,7.039116,...,12.159326,20.200807,21.232718,20.755683,17.574028,16.839528,16.564554,17.059779,17.884833,10.751563
min,41.000000,47.000000,49.000000,9.000000e+03,500.000000,16.000000,155.000000,49.000000,1.000000,1.000000,...,12.000000,4.000000,5.000000,5.000000,2.000000,2.000000,2.000000,2.000000,2.000000,15.000000
25%,214413.500000,61.000000,67.000000,4.750000e+05,1000.000000,21.000000,176.000000,70.000000,1.000000,7.000000,...,50.000000,29.000000,28.000000,25.000000,8.000000,8.000000,8.000000,8.000000,8.000000,27.000000
50%,236543.000000,66.000000,71.000000,9.750000e+05,3000.000000,25.000000,181.000000,75.000000,1.000000,12.000000,...,59.000000,52.000000,56.000000,53.000000,11.000000,11.000000,11.000000,11.000000,11.000000,36.000000
75%,253532.500000,70.000000,75.000000,2.000000e+06,8000.000000,29.000000,186.000000,80.000000,1.000000,19.000000,...,66.000000,63.000000,65.000000,63.000000,14.000000,14.000000,14.000000,14.000000,14.000000,45.000000
max,264640.000000,93.000000,95.000000,1.940000e+08,350000.000000,54.000000,206.000000,110.000000,5.000000,28.000000,...,96.000000,93.000000,93.000000,92.000000,91.000000,92.000000,93.000000,92.000000,90.000000,65.000000


In [10]:
fifa_22.columns

Index(['sofifa_id', 'short_name', 'overall', 'potential', 'value_eur',
       'wage_eur', 'age', 'height_cm', 'weight_kg', 'club_name', 'league_name',
       'league_level', 'club_position', 'nationality_name', 'nation_position',
       'nation_jersey_number', 'preferred_foot', 'weak_foot', 'skill_moves',
       'international_reputation', 'work_rate', 'body_type',
       'release_clause_eur', 'pace', 'shooting', 'passing', 'dribbling',
       'defending', 'physic', 'attacking_crossing', 'attacking_finishing',
       'attacking_heading_accuracy', 'attacking_short_passing',
       'attacking_volleys', 'skill_dribbling', 'skill_curve',
       'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control',
       'movement_acceleration', 'movement_sprint_speed', 'movement_agility',
       'movement_reactions', 'movement_balance', 'power_shot_power',
       'power_jumping', 'power_stamina', 'power_strength', 'power_long_shots',
       'mentality_aggression', 'mentality_interceptions',
   

In [11]:
#What if key attacker, midfielder, defender leaves the team
player_attributes_cols = ['sofifa_id','short_name','work_rate','club_position','preferred_foot','overall','potential','value_eur','wage_eur','age','height_cm','weight_kg','weak_foot','skill_moves',
       'international_reputation','release_clause_eur', 'pace', 'shooting', 'passing', 'dribbling',
       'defending', 'physic', 'attacking_crossing', 'attacking_finishing',
       'attacking_heading_accuracy', 'attacking_short_passing',
       'attacking_volleys', 'skill_dribbling', 'skill_curve',
       'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control',
       'movement_acceleration', 'movement_sprint_speed', 'movement_agility',
       'movement_reactions', 'movement_balance', 'power_shot_power',
       'power_jumping', 'power_stamina', 'power_strength', 'power_long_shots',
       'mentality_aggression', 'mentality_interceptions',
       'mentality_positioning', 'mentality_vision', 'mentality_penalties',
       'mentality_composure']

In [12]:
player_attributes = fifa_22[player_attributes_cols]

In [13]:
k = player_attributes.iloc[0]['work_rate']
k

'Medium/Low'

In [14]:
def wr_manipulation(wr):
    s=""
    for i in wr:
        if i=='/':
            break
        s=s+i
    return s    

In [15]:
wr_manipulation(k)

'Medium'

In [16]:
player_attributes['work_rate'] = player_attributes['work_rate'].apply(lambda x : wr_manipulation(x))

In [17]:
player_attributes.head()

,sofifa_id,short_name,work_rate,club_position,preferred_foot,overall,potential,value_eur,wage_eur,age,...,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure
0,158023,L. Messi,Medium,RW,Left,93,93,78000000.0,320000.0,34,...,68,72,69,94,44,40,93,95,75,96
1,188545,R. Lewandowski,High,ST,Right,92,92,119500000.0,270000.0,32,...,85,76,86,87,81,49,95,81,90,88
2,20801,Cristiano Ronaldo,High,ST,Right,91,91,45000000.0,270000.0,36,...,95,77,77,93,63,29,95,76,88,95
3,190871,Neymar Jr,High,LW,Right,91,91,129000000.0,270000.0,29,...,64,81,53,81,63,37,86,90,93,93
4,192985,K. De Bruyne,High,RCM,Right,91,91,125500000.0,350000.0,30,...,63,89,74,91,76,66,88,94,83,89


In [18]:
le = preprocessing.LabelEncoder()
player_attributes['work_rate'] = le.fit_transform(player_attributes['work_rate'])
player_attributes['preferred_foot'] = le.fit_transform(player_attributes['preferred_foot'])
player_attributes['club_position'] = le.fit_transform(player_attributes['club_position'])

In [19]:
player_attributes.head()

,sofifa_id,short_name,work_rate,club_position,preferred_foot,overall,potential,value_eur,wage_eur,age,...,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure
0,158023,L. Messi,2,25,0,93,93,78000000.0,320000.0,34,...,68,72,69,94,44,40,93,95,75,96
1,188545,R. Lewandowski,0,27,1,92,92,119500000.0,270000.0,32,...,85,76,86,87,81,49,95,81,90,88
2,20801,Cristiano Ronaldo,0,27,1,91,91,45000000.0,270000.0,36,...,95,77,77,93,63,29,95,76,88,95
3,190871,Neymar Jr,0,14,1,91,91,129000000.0,270000.0,29,...,64,81,53,81,63,37,86,90,93,93
4,192985,K. De Bruyne,0,19,1,91,91,125500000.0,350000.0,30,...,63,89,74,91,76,66,88,94,83,89


In [20]:
player_attributes.columns

Index(['sofifa_id', 'short_name', 'work_rate', 'club_position',
       'preferred_foot', 'overall', 'potential', 'value_eur', 'wage_eur',
       'age', 'height_cm', 'weight_kg', 'weak_foot', 'skill_moves',
       'international_reputation', 'release_clause_eur', 'pace', 'shooting',
       'passing', 'dribbling', 'defending', 'physic', 'attacking_crossing',
       'attacking_finishing', 'attacking_heading_accuracy',
       'attacking_short_passing', 'attacking_volleys', 'skill_dribbling',
       'skill_curve', 'skill_fk_accuracy', 'skill_long_passing',
       'skill_ball_control', 'movement_acceleration', 'movement_sprint_speed',
       'movement_agility', 'movement_reactions', 'movement_balance',
       'power_shot_power', 'power_jumping', 'power_stamina', 'power_strength',
       'power_long_shots', 'mentality_aggression', 'mentality_interceptions',
       'mentality_positioning', 'mentality_vision', 'mentality_penalties',
       'mentality_composure'],
      dtype='object')

In [21]:
id_name = fifa_22[['short_name','sofifa_id']].set_index('sofifa_id')['short_name'].to_dict()
id_name

{158023: 'L. Messi',
 188545: 'R. Lewandowski',
 20801: 'Cristiano Ronaldo',
 190871: 'Neymar Jr',
 192985: 'K. De Bruyne',
 200389: 'J. Oblak',
 231747: 'K. Mbappé',
 167495: 'M. Neuer',
 192448: 'M. ter Stegen',
 202126: 'H. Kane',
 215914: 'N. Kanté',
 165153: 'K. Benzema',
 192119: 'T. Courtois',
 200104: 'H. Son',
 200145: 'Casemiro',
 203376: 'V. van Dijk',
 208722: 'S. Mané',
 209331: 'M. Salah',
 210257: 'Ederson',
 212622: 'J. Kimmich',
 212831: 'Alisson',
 230621: 'G. Donnarumma',
 155862: 'Sergio Ramos',
 176580: 'L. Suárez',
 182521: 'T. Kroos',
 192505: 'R. Lukaku',
 193041: 'K. Navas',
 202652: 'R. Sterling',
 212198: 'Bruno Fernandes',
 239085: 'E. Haaland',
 153079: 'S. Agüero',
 167948: 'H. Lloris',
 177003: 'L. Modrić',
 183898: 'Á. Di María',
 186153: 'W. Szczęsny',
 189596: 'T. Müller',
 192387: 'C. Immobile',
 195864: 'P. Pogba',
 199556: 'M. Verratti',
 207865: 'Marquinhos',
 209658: 'L. Goretzka',
 211110: 'P. Dybala',
 216267: 'A. Robertson',
 228702: 'F. de Jon

In [22]:
NUM_RECOM = 10
X = player_attributes.iloc[:,2:].dropna().values
nbrs = NearestNeighbors(n_neighbors=NUM_RECOM+1, algorithm='ball_tree').fit(X)
dist, rank = nbrs.kneighbors(X)

In [23]:
rank.shape

(16020, 11)

In [24]:
dist.shape

(16020, 11)

In [25]:
player_cols = ['work_rate', 'club_position', 'preferred_foot', 'overall', 'potential',
       'value_eur', 'wage_eur', 'age', 'height_cm', 'weight_kg', 'weak_foot',
       'skill_moves', 'international_reputation', 'release_clause_eur', 'pace',
       'shooting', 'passing', 'dribbling', 'defending', 'physic',
       'attacking_crossing', 'attacking_finishing',
       'attacking_heading_accuracy', 'attacking_short_passing',
       'attacking_volleys', 'skill_dribbling', 'skill_curve',
       'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control',
       'movement_acceleration', 'movement_sprint_speed', 'movement_agility',
       'movement_reactions', 'movement_balance', 'power_shot_power',
       'power_jumping', 'power_stamina', 'power_strength', 'power_long_shots',
       'mentality_aggression', 'mentality_interceptions',
       'mentality_positioning', 'mentality_vision', 'mentality_penalties',
       'mentality_composure']

In [26]:
play = fifa_22[['sofifa_id','short_name'] + player_cols].dropna()
play.head()

,sofifa_id,short_name,work_rate,club_position,preferred_foot,overall,potential,value_eur,wage_eur,age,...,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure
0,158023,L. Messi,Medium/Low,RW,Left,93,93,78000000.0,320000.0,34,...,68,72,69,94,44,40,93,95,75,96
1,188545,R. Lewandowski,High/Medium,ST,Right,92,92,119500000.0,270000.0,32,...,85,76,86,87,81,49,95,81,90,88
2,20801,Cristiano Ronaldo,High/Low,ST,Right,91,91,45000000.0,270000.0,36,...,95,77,77,93,63,29,95,76,88,95
3,190871,Neymar Jr,High/Medium,LW,Right,91,91,129000000.0,270000.0,29,...,64,81,53,81,63,37,86,90,93,93
4,192985,K. De Bruyne,High/High,RCM,Right,91,91,125500000.0,350000.0,30,...,63,89,74,91,76,66,88,94,83,89


In [27]:
similar_df = pd.DataFrame(columns = [f'rank_{i}'for i in range(1,NUM_RECOM+1)],
                          index = play['sofifa_id'].values,
                          data=rank[:,1:])
#columns will start from rank_1 and go till rank_11
#index of the datafrmae is the fifa id of the player
#
dist_df = pd.DataFrame(columns=[f'rank_{i}'for i in range(1,NUM_RECOM+1)],
                       index=play['sofifa_id'].values,
                       data=dist[:,1:])

In [28]:
similar_df

,rank_1,rank_2,rank_3,rank_4,rank_5,rank_6,rank_7,rank_8,rank_9,rank_10
158023,27,28,51,78,49,52,77,50,53,17
188545,20,19,9,14,36,12,13,34,7,119
20801,415,66,239,154,232,230,410,96,164,150
190871,4,6,21,33,35,34,19,20,1,9
192985,3,35,6,21,34,33,19,20,1,9
...,...,...,...,...,...,...,...,...,...,...
261962,15953,15976,15992,15848,15977,15991,15905,15829,15995,15994
262040,15912,15999,15730,15929,11285,15874,15971,15838,15909,16005
262760,15855,15886,15877,15869,15878,15887,15921,15866,15987,16004
262820,15884,16003,16013,15922,15875,16014,15946,15958,15965,15901


In [29]:
for cols in list(similar_df):
    tg_col = similar_df[cols]
    new_value = play['sofifa_id'].iloc[tg_col].tolist()
    similar_df[cols] = new_value

In [30]:
similar_df

,rank_1,rank_2,rank_3,rank_4,rank_5,rank_6,rank_7,rank_8,rank_9,rank_10
158023,195864,199556,212218,231677,209499,213345,231478,210514,218667,182521
188545,212198,202652,200104,212622,239818,208722,209331,231281,215914,235790
20801,237383,188567,244260,220651,234943,234378,232293,203263,230658,213648
190871,192985,202126,239085,228702,233049,231281,202652,212198,188545,200104
192985,190871,233049,202126,239085,231281,228702,202652,212198,188545,200104
...,...,...,...,...,...,...,...,...,...,...
261962,261956,257934,261932,257493,258987,261890,243084,243107,261957,261939
262040,252756,262034,245417,257690,169584,261981,252332,253746,248376,243790
262760,259303,263039,262123,261750,262127,263261,256414,261301,261666,243725
262820,262651,264150,261667,256415,262020,261818,261751,262125,264142,264500


In [31]:
def similar_player(similar_df, dist_df, player_id):
    player_id = int(player_id)
    player_name = play[play['sofifa_id'] == player_id]['short_name'].values[0]
    
    Xaxis = 1/(1+dist_df.loc[player_id].values[::-1])
    Yaxis = similar_df.loc[player_id].map(id_name).values[::-1]
    
    fig = go.Figure(go.Bar(
                x=Xaxis,
                y=Yaxis,
                orientation='h'))
    fig.update_layout(title_text='Players similar to  " '+str(player_name)+' "')
    fig.show()
    
    sim_ply = similar_df.loc[player_id]
    display_col = ['short_name','age','overall', 'club_position', 
                   'preferred_foot','weak_foot','pace',
       'shooting', 'passing', 'dribbling']

    display_df = pd.DataFrame({'sofifa_id':similar_df.loc[player_id]})\
    .merge(play[['sofifa_id']+display_col], how='left', on='sofifa_id')[display_col]


    display(display_df.style.highlight_max(color='lightgreen').highlight_min(color='#cd4f39'))
    

In [38]:
similar_player(similar_df, dist_df,153079)

,short_name,age,overall,club_position,preferred_foot,weak_foot,pace,shooting,passing,dribbling
0,E. Hazard,30,85,LW,Right,4,84.000000,80.000000,82.000000,88.000000
1,M. Ødegaard,22,82,CAM,Left,2,77.000000,74.000000,83.000000,84.000000
2,Koke,29,85,RCM,Right,4,70.000000,75.000000,85.000000,81.000000
3,Gabriel Jesus,24,83,SUB,Right,3,84.000000,81.000000,72.000000,86.000000
4,Marco Asensio,25,83,SUB,Left,3,82.000000,80.000000,81.000000,83.000000
5,Á. Correa,26,83,SUB,Right,4,85.000000,83.000000,78.000000,87.000000
6,T. Lemar,25,83,LCM,Left,3,82.000000,76.000000,82.000000,86.000000
7,Merino,25,83,RCM,Left,3,74.000000,78.000000,81.000000,82.000000
8,Vinícius Jr.,20,80,SUB,Right,4,95.000000,70.000000,71.000000,87.000000
9,C. Pulisic,22,82,LW,Right,4,89.000000,70.000000,73.000000,86.000000
